<a href="https://colab.research.google.com/github/ojcharles/snippets/blob/main/oscars_paper_clustering_by_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scientific Article Literature Clustering by NLP

Oscar Charles 2021

This is a notebook that clusters journals, and can help you find similar papers to those you already know are important.

What you need:


*   A pubmed query. Example included
*   A list of DOI's for papers you already care about. Example included



Question: Can we use Natural Language processing to take a set of abstracts (strings) -> tokens -> vectorisation -> clustering of abstracts by similarity? We have a set of papers we want to cluster close together, can we get them too such that we can find similar papers we may have missed?



In [ ]:
# install all the python stuff I need
# get the requirements file
!wget https://raw.githubusercontent.com/ojcharles/snippets/main/nlp_lit_clustering_requirements.txt

# install the packages
!pip install -r nlp_lit_clustering_requirements.txt

# install the spacy model
# A full spaCy pipeline for biomedical data with a larger vocabulary and 50k word vectors.
! python -m pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_core_sci_md-0.5.0.tar.gz

In [ ]:
# also install EDirect for pubmed APi calls
!  yes | sh -c "$(curl -fsSL ftp://ftp.ncbi.nlm.nih.gov/entrez/entrezdirect/install-edirect.sh)"


Entrez Direct has been successfully downloaded and installed.


To activate EDirect for this terminal session, please execute the following:

export PATH=${PATH}:${HOME}/edirect



Getting paper data from Pubmed and parseing the json

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import xml.etree.ElementTree as ET
import bokeh

In [ ]:
# pubmed has all papers, epmc only those that are openly accessible.

# Conduct a PubMed search and retrieve the results as a list of PMIDs
! ${HOME}/edirect/esearch -db pubmed -query "((resistance)) AND ((ribavirin) OR (favipiravir) OR (remdesivir) OR (EIDD-2801) OR (molnupiravir)) "   | ${HOME}/edirect/efetch -format uid > pubmed.txt

# how many entries?
! wc -l pubmed.txt 

# take those identified pubmed ids of interest and 
! ${HOME}/edirect/efetch -db pubmed -input pubmed.txt -format xml > pubmed.xml

# sanity check
! head -5 pubmed.xml

1807 pubmed.txt
<?xml version="1.0" encoding="UTF-8" ?>
<!DOCTYPE PubmedArticleSet>
<PubmedArticleSet>
  <PubmedArticle>
    <MedlineCitation Status="PubMed-not-MEDLINE" Owner="NLM">


In [ ]:
# parse xml
dict_ = {'doi': [], 'title': [], 'abstract':[]}

tree = ET.parse('pubmed.xml') # download from a search in epmc
root = tree.getroot()

search_doi = 'PubmedData/ArticleIdList/ArticleId[@IdType="doi"]'
search_title = 'MedlineCitation/Article/ArticleTitle'
search_abstract = 'MedlineCitation/Article/Abstract/AbstractText'

for entry in root.iter('PubmedArticle'):
    if entry.find(search_doi) is None:
        continue
    else:
        doi = entry.find(search_doi).text
        
    if entry.find(search_title) is None:
        continue
    else:
        title = entry.find(search_title).text
        
    if entry.find(search_abstract) is None:
        continue
    else:
        abstractText = entry.find(search_abstract).text
    #print(doi, title)
    dict_['doi'].append(doi)
    dict_['title'].append(title)
    dict_['abstract'].append(abstractText)


In [ ]:
# format the dict to a table
df = pd.DataFrame(dict_, columns=['doi', 'title','abstract'])
df.head()

,doi,title,abstract
0,10.2147/IDR.S353605,Identification of NS5B Resistance-Associated M...,Treatment of HCV infection with peginterferon ...
1,16230,Management of SARS-CoV-2 infection: recommenda...,The first Polish recommendations regarding the...
2,527,Genetic Diversity Does Not Contribute to Atten...,The disease yellow fever was prevented by two ...
3,10.1038/s41467-022-29104-y,De novo emergence of a remdesivir resistance m...,SARS-CoV-2 remdesivir resistance mutations hav...
4,S0166-3542(22)00042-0,High dose sofosbuvir and sofosbuvir-plus-ribav...,Hepatitis E virus (HEV) is an important cause ...


In [ ]:
# how many papers are left with al the required info?
len(df)

1301

## Paper Encoding

Here we take the text and identify each token "word" and use the spacy model to find the relationships etc.

In [ ]:
#NLP 
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
# spacy.prefer_gpu() # if you have a GPU do this its so much faster

# using scispacy we can add support for scientific abbreviations
from scispacy.abbreviation import AbbreviationDetector

parser = spacy.load("en_core_sci_md") # scispacy model
# Add the abbreviation pipe to the spacy pipeline.
parser.add_pipe("abbreviation_detector")


/usr/local/lib/python3.7/dist-packages/spacy/util.py:833: UserWarning: [W095] Model 'en_core_sci_md' (0.5.0) was trained with spaCy v3.2 and may not be 100% compatible with the current version (3.2.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [ ]:
import string

punctuations = string.punctuation
stopwords = list(STOP_WORDS)
stopwords[:10]

['besides',
 'did',
 'there',
 'first',
 'some',
 'nobody',
 'whole',
 'front',
 'their',
 '‘ll']

In [ ]:
# stop words are words that are removed.
# as We don't want a cluster per virus, per drug etc but generally paper with novel rna resmuts we add these extra stopwords.
custom_stop_words = [
    'doi', 'preprint', 'copyright', 'peer', 'reviewed', 'org', 'https', 'et', 'al', 'author', 'figure', 
    'rights', 'reserved', 'permission', 'used', 'using', 'biorxiv', 'medrxiv', 'license', 'fig', 'fig.', 
    'al.', 'Elsevier', 'PMC', 'CZI', 'www',
    "Hepatitis C", "Herpes", "Simplex 1", "Simplex 2", "Cytomegalovirus", "Coronavirus", "HCMV", "CMV", "HSV1", "HSV2", 
    "influenza A", "(H1N1)", "HIV-1", "HIV", "Hepatitis B", "HCV", "HBV",
    "SARS-COV-2", "COVID-19", "COVID", "human immunodeficiency virus", "HIV-RNA", "subtype", "genotype", "subtype",
    "avian influenza", "syncytial", "polio", "poliovirus", "chikungunya"
    
]

for w in custom_stop_words:
    if w not in stopwords:
        stopwords.append(w)

In [ ]:
# Parser

parser.max_length = 7000000

def call_tokenizer(df):
    df["processed_abstract"] = df["abstract"].apply(spacy_tokenizer)
    return df

def spacy_tokenizer(sentence):
    mytokens = parser(sentence)
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    mytokens = [ word for word in mytokens if word not in stopwords and word not in punctuations ]
    mytokens = " ".join([i for i in mytokens])
    return mytokens

In [ ]:
from multiprocessing import  Pool

def parallelize_dataframe(df, func, n_cores=4):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [ ]:
%%capture --no-stdout
# scispacy abbreviation complains each time a abstract has no abbreviations it knows.... stop that warning.
# tokenise
# single core - single gpu
df["processed_abstract"] = df["abstract"].apply(spacy_tokenizer)
# many cpu
#%time df = parallelize_dataframe(df=df, func=call_tokenizer, n_cores=16)

### save


In [ ]:
import pickle
pickle.dump(df, open("tokenised_df.p", "wb" ))

## paper vectoriser


In [ ]:
df = pickle.load(open("tokenised_df.p", "rb"))
#df = df.sample(600000)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
def vectorize(text, maxx_features):
    
    vectorizer = TfidfVectorizer(max_features=maxx_features)
    X = vectorizer.fit_transform(text)
    return X

In [ ]:
text = df['processed_abstract'].values
%time X = vectorize(text, 2 ** 12)
X.shape

CPU times: user 122 ms, sys: 0 ns, total: 122 ms
Wall time: 123 ms


(1301, 4096)

##Cluster

In [ ]:
# as well as x and y space, lets colour each paper by Kmeans clustering
from sklearn.cluster import KMeans
k = 20
kmeans = KMeans(n_clusters=k, random_state=42)
%time y_pred = kmeans.fit_predict(X)
df['y'] = y_pred

CPU times: user 1.49 s, sys: 105 ms, total: 1.6 s
Wall time: 1.78 s


## Plotting

In [ ]:
# reduce dimensions
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA

pca = PCA(n_components=30, random_state=42)
%time X_embedded_pca= pca.fit_transform(X.toarray())
X_embedded_pca.shape

CPU times: user 903 ms, sys: 122 ms, total: 1.02 s
Wall time: 557 ms


(1301, 30)

In [ ]:
# t-sne
from sklearn.manifold import TSNE
X_embedded_tsne = TSNE(n_components=2, learning_rate='auto',init='random').fit_transform(X_embedded_pca)
df['x_tsne'] = X_embedded_tsne[:,0] 
df['y_tsne'] = X_embedded_tsne[:,1]

#from tsnecuda import TSNE
#%time X_embedded_tsne = TSNE(n_components=2, learning_rate=30, n_iter=500000, verbose=1).fit_transform(X_embedded_pca)

In [ ]:
# identify which papers we know are of interest, then make them squares. maybe we need to hide the virus name from the model?
key_doi=["10.1371/journal.ppat.1009929",
"10.1128/mBio.00221-18",
"10.1128/JVI.01965-17",
"10.1073/pnas.1811345115",
"10.1093/jac/dku209",
"10.1128/AAC.01073-16",
"10.1073/pnas.1232294100",
"10.1128/JVI.02139-12",
"10.1016/j.antiviral.2013.07.008",
"10.1371/journal.ppat.1001163",
"10.1128/JVI.00289-14",
"10.1073/pnas.1111650108",
"10.1128/JVI.79.4.2346-2355.2005",
"10.1128/JVI.03594-13",
"10.1128/JVI.01297-08",
"10.1371/journal.ppat.1003877",
"10.1128/JVI.01528-14",
        "10.1128/JVI.00367-19",
        "10.1128/JVI.00078-16",
        "10.1016/j.jmii.2017.03.004",
        "10.1038/ncomms5794",
        "10.1074/jbc.C112.401471",
        "10.1099/jgv.0.000316",
        "10.3791/2953",
        "10.1099/jgv.0.000682",
        "10.1371/journal.ppat.1001163",
        "10.1016/j.bbrc.2013.12.071",
        "10.1038/nm1726",
        "10.1371/journal.ppat.1005010"]
df['key'] = df['doi'].isin(key_doi)
df[df['doi'].isin(key_doi)]

,doi,title,abstract,processed_abstract,y,x_tsne,y_tsne,key
35,10.1371/journal.ppat.1009929,In vitro selection of Remdesivir resistance su...,"Remdesivir (RDV), a broadly acting nucleoside ...",remdesivir rdv broadly act nucleoside analogue...,6,-11.084248,45.179966,True
229,10.1073/pnas.1811345115,The mechanism of resistance to favipiravir in ...,Favipiravir is a broad-spectrum antiviral that...,favipiravir broad-spectrum antiviral promise t...,18,-35.909420,21.598940,True
384,10.1099/jgv.0.000682,Mutagen resistance and mutation restriction of...,The error rate of the RNA-dependent RNA polyme...,error rate rna-dependent rna polymerase rdrp r...,8,-17.598848,25.597475,True
443,10.1128/JVI.00078-16,Poliovirus Polymerase Leu420 Facilitates RNA R...,RNA recombination is important in the formatio...,rna recombination important formation picornav...,8,-15.428197,24.191156,True
449,10.1128/AAC.01073-16,In Vitro Assessment of Combinations of Enterov...,Enterovirus 71 (EV-A71) is a major causative p...,enterovirus 71 ev-a71 major causative pathogen...,8,-12.135286,18.616566,True
539,10.1099/jgv.0.000316,Amino acid residues Ala283 and His421 in the R...,The quasispecies diversity of RNA viruses is m...,quasispecie diversity rna virus mainly determi...,8,-16.780453,25.325102,True
676,10.1038/ncomms5794,Generation and characterization of influenza A...,Genetic diversity of influenza A viruses (IAV)...,genetic diversity influenza virus iav acquire ...,8,-18.304201,23.493374,True
702,10.1093/jac/dku209,Mutations in the chikungunya virus non-structu...,"T-705, also known as favipiravir, is a small-m...",t-705 know favipiravir small-molecule inhibito...,18,-38.347027,22.072716,True
743,10.1128/JVI.00289-14,Attenuation of human enterovirus 71 high-repli...,"In a screen for ribavirin resistance, a novel ...",screen ribavirin resistance novel high-fidelit...,8,-17.242252,24.193941,True
758,10.1128/JVI.03594-13,Ribavirin-resistant variants of foot-and-mouth...,Mutagenic nucleoside analogues can be used to ...,mutagenic nucleoside analogue use isolate rna ...,8,-18.198774,26.157326,True


In [ ]:
# we want bokeh to treat y (cluster) as discrete not continuous 
df['y'] = df['y'].apply(str)
df.dtypes

doi                    object
title                  object
abstract               object
processed_abstract     object
y                      object
x_tsne                float32
y_tsne                float32
key                      bool
dtype: object

In [ ]:
# make a pretty plot
from bokeh.plotting import ColumnDataSource, figure, output_notebook, show, output_file, save
from bokeh.palettes import d3
import bokeh.models as bmo
#output_file("toolbar.html")
output_notebook()

TOOLTIPS = [
    ("index", "$index"),
    ("(x,y)", "($x, $y)"),
    ("doi", "@doi"),
    ("title", "@title"),
]

# use whatever palette you want...
palette = d3['Category20'][len(df['y'].unique())]
color_map = bmo.CategoricalColorMapper(factors=df['y'].unique(),
                                   palette=palette)

p = figure(width=800, height=800, tooltips=TOOLTIPS,
           title="Mouse over the dots")
p.scatter('x_tsne', 'y_tsne', size=5, source=df[df['key'] == False], color={'field': 'y', 'transform': color_map})
p.scatter('x_tsne', 'y_tsne', size=8, source=df[df['key'] == True], color="black", marker="square")
show(p)
